In [22]:
# Part 1: First run the FABP4 data collection and processing
# [Previous code up to getting df_final remains the same]

# Part 2: PaDEL descriptor calculation
import os
import urllib.request
import pandas as pd

def download_file(url, filename):
    try:
        urllib.request.urlretrieve(url, filename)
        print(f"Successfully downloaded {filename}")
    except Exception as e:
        print(f"Error downloading {filename}: {e}")

# Download required files
padel_zip_url = "https://github.com/dataprofessor/bioinformatics/raw/master/padel.zip"
padel_sh_url = "https://github.com/dataprofessor/bioinformatics/raw/master/padel.sh"

if not os.path.exists('padel.zip'):
    download_file(padel_zip_url, 'padel.zip')
if not os.path.exists('padel.sh'):
    download_file(padel_sh_url, 'padel.sh')

# Unzip padel
if os.path.exists('padel.zip'):
    ! unzip -o padel.zip
    print("Unzipped padel.zip")
else:
    print("padel.zip not found")

# Verify we have our processed data
if 'df_final' not in globals():
    print("Loading previously saved data...")
    df_final = pd.read_csv('FABP4_04_bioactivity_data_3class_pIC50.csv')

# Prepare data for PaDEL descriptor calculation
selection = ['canonical_smiles', 'molecule_chembl_id']
df_selection = df_final[selection]
df_selection.to_csv('molecule.smi', sep='\t', index=False, header=False)

print("First 5 molecules:")
with open('molecule.smi', 'r') as f:
    for i, line in enumerate(f):
        if i < 5:
            print(line.strip())

print("\nTotal number of molecules:")
with open('molecule.smi', 'r') as f:
    print(len(f.readlines()))

# Make padel.sh executable
! chmod +x padel.sh

# Calculate PaDEL descriptors
print("\nCalculating PaDEL descriptors...")
! ./padel.sh

# Load descriptor data and prepare X matrix
if os.path.exists('FABP4_descriptors_rdkit.csv.csv'):
    df_X = pd.read_csv('FABP4_descriptors_rdkit.csv')
    df_X = df_X.drop(columns=['Name'])

    # Prepare Y variable (pIC50)
    df_Y = df_final['pIC50']

    # Combine X and Y into final dataset
    dataset_final = pd.concat([df_X, df_Y], axis=1)

    # Save final dataset
    dataset_final.to_csv('FABP4_06_bioactivity_data_3class_pIC50_pubchem_fp.csv', index=False)

    print("\nPaDEL descriptor analysis completed!")
    print("Final dataset saved as: FABP4_06_bioactivity_data_3class_pIC50_pubchem_fp.csv")
else:
    print("Error: descriptors_output.csv not found")

Archive:  padel.zip
  inflating: __MACOSX/._PaDEL-Descriptor  
  inflating: PaDEL-Descriptor/MACCSFingerprinter.xml  
  inflating: __MACOSX/PaDEL-Descriptor/._MACCSFingerprinter.xml  
  inflating: PaDEL-Descriptor/AtomPairs2DFingerprinter.xml  
  inflating: __MACOSX/PaDEL-Descriptor/._AtomPairs2DFingerprinter.xml  
  inflating: PaDEL-Descriptor/EStateFingerprinter.xml  
  inflating: __MACOSX/PaDEL-Descriptor/._EStateFingerprinter.xml  
  inflating: PaDEL-Descriptor/Fingerprinter.xml  
  inflating: __MACOSX/PaDEL-Descriptor/._Fingerprinter.xml  
  inflating: PaDEL-Descriptor/.DS_Store  
  inflating: __MACOSX/PaDEL-Descriptor/._.DS_Store  
  inflating: __MACOSX/PaDEL-Descriptor/._license  
  inflating: PaDEL-Descriptor/KlekotaRothFingerprintCount.xml  
  inflating: __MACOSX/PaDEL-Descriptor/._KlekotaRothFingerprintCount.xml  
  inflating: PaDEL-Descriptor/config  
  inflating: __MACOSX/PaDEL-Descriptor/._config  
  inflating: PaDEL-Descriptor/PubchemFingerprinter.xml  
  inflating: __MAC

In [16]:
! pip install rdkit

In [26]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors, DataStructs
from typing import Optional, Dict

class MolecularDescriptorCalculator:
    """Calculate molecular descriptors using RDKit."""
    
    def create_mol_from_smiles(self, smiles: str) -> Optional[Chem.Mol]:
        try:
            mol = Chem.MolFromSmiles(smiles)
            if mol is None:
                logger.warning(f"Could not parse SMILES: {smiles}")
            return mol
        except Exception as e:
            logger.error(f"Error creating molecule from SMILES {smiles}: {e}")
            return None

    def calculate_fingerprints(self, mol: Chem.Mol) -> np.ndarray:
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, 3, nBits=2048)
        array = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(fp, array)
        return array

    def calculate_descriptors(self, mol: Chem.Mol) -> Dict[str, float]:
        descriptor_functions = {
            'MW': Descriptors.ExactMolWt,
            'LogP': Descriptors.MolLogP,
            'TPSA': Descriptors.TPSA,
            'HBA': Descriptors.NumHAcceptors,
            'HBD': Descriptors.NumHDonors,
            'RotableBonds': Descriptors.NumRotatableBonds,
            'AromaticRings': Descriptors.NumAromaticRings,
            'HeavyAtoms': Descriptors.HeavyAtomCount,
            'Rings': Descriptors.RingCount
        }
        
        descriptors = {}
        for name, func in descriptor_functions.items():
            try:
                descriptors[name] = func(mol)
            except:
                descriptors[name] = np.nan
        return descriptors

    def process_molecules(self, df: pd.DataFrame) -> Optional[pd.DataFrame]:
        if 'canonical_smiles' not in df.columns:
            logger.error("'canonical_smiles' column not found in DataFrame")
            return None

        results = []
        n_mols = len(df)
        
        for idx, row in df.iterrows():
            if (idx + 1) % 50 == 0:
                logger.info(f"Processing molecule {idx + 1} of {n_mols}")
                
            smiles = row['canonical_smiles']
            mol = self.create_mol_from_smiles(smiles)
            
            if mol is None:
                continue
                
            # Calculate fingerprints
            fp_array = self.calculate_fingerprints(mol)
            fp_dict = {f'fp_{i}': int(x) for i, x in enumerate(fp_array)}
            
            # Calculate other descriptors
            descriptors = self.calculate_descriptors(mol)
            
            # Combine all features
            result = {'smiles': smiles, **fp_dict, **descriptors}
            results.append(result)

        if not results:
            logger.error("No valid molecules processed")
            return None

        # Create output DataFrame
        df_descriptors = pd.DataFrame(results)
        
        # Add pIC50 if present in original DataFrame
        if 'pIC50' in df.columns:
            df_descriptors = pd.merge(
                df_descriptors, 
                df[['canonical_smiles', 'pIC50']], 
                left_on='smiles', 
                right_on='canonical_smiles'
            )
            df_descriptors = df_descriptors.drop('canonical_smiles', axis=1)

        return df_descriptors

# Verify we have our processed data
if 'df_final' not in globals():
    print("Loading previously saved data...")
    df_final = pd.read_csv('FABP4_04_bioactivity_data_3class_pIC50.csv')

# Initialize calculator and process molecules
calculator = MolecularDescriptorCalculator()
print("\nCalculating molecular descriptors...")
result_df = calculator.process_molecules(df_final)

if result_df is not None:
    # Save final dataset
    output_file = 'FABP4_06_bioactivity_data_3class_pIC50_rdkit_fp.csv'
    result_df.to_csv(output_file, index=False)
    print("\nMolecular descriptor analysis completed!")
    print(f"Final dataset saved as: {output_file}")
    print(f"Dataset shape: {result_df.shape}")
    print(f"Number of features: {len(result_df.columns) - 2}")  # -2 for smiles and pIC50
else:
    print("Error: Failed to process molecules")

[09:18:00] DEPRECATION WARNING: please use MorganGenerator
[09:18:00] DEPRECATION WARNING: please use MorganGenerator
[09:18:00] DEPRECATION WARNING: please use MorganGenerator
[09:18:00] DEPRECATION WARNING: please use MorganGenerator
[09:18:00] DEPRECATION WARNING: please use MorganGenerator
[09:18:00] DEPRECATION WARNING: please use MorganGenerator
[09:18:00] DEPRECATION WARNING: please use MorganGenerator
[09:18:00] DEPRECATION WARNING: please use MorganGenerator
[09:18:00] DEPRECATION WARNING: please use MorganGenerator
[09:18:00] DEPRECATION WARNING: please use MorganGenerator
[09:18:00] DEPRECATION WARNING: please use MorganGenerator
[09:18:00] DEPRECATION WARNING: please use MorganGenerator
[09:18:00] DEPRECATION WARNING: please use MorganGenerator
[09:18:00] DEPRECATION WARNING: please use MorganGenerator
[09:18:00] DEPRECATION WARNING: please use MorganGenerator
[09:18:00] DEPRECATION WARNING: please use MorganGenerator
[09:18:00] DEPRECATION WARNING: please use MorganGenerat


Calculating molecular descriptors...


[09:18:00] DEPRECATION WARNING: please use MorganGenerator
[09:18:00] DEPRECATION WARNING: please use MorganGenerator
[09:18:00] DEPRECATION WARNING: please use MorganGenerator
[09:18:00] DEPRECATION WARNING: please use MorganGenerator
[09:18:00] DEPRECATION WARNING: please use MorganGenerator
[09:18:00] DEPRECATION WARNING: please use MorganGenerator
[09:18:00] DEPRECATION WARNING: please use MorganGenerator
[09:18:00] DEPRECATION WARNING: please use MorganGenerator
[09:18:00] DEPRECATION WARNING: please use MorganGenerator
[09:18:00] DEPRECATION WARNING: please use MorganGenerator
[09:18:00] DEPRECATION WARNING: please use MorganGenerator
[09:18:00] DEPRECATION WARNING: please use MorganGenerator
[09:18:00] DEPRECATION WARNING: please use MorganGenerator
[09:18:00] DEPRECATION WARNING: please use MorganGenerator
[09:18:00] DEPRECATION WARNING: please use MorganGenerator
[09:18:00] DEPRECATION WARNING: please use MorganGenerator
[09:18:00] DEPRECATION WARNING: please use MorganGenerat


Molecular descriptor analysis completed!
Final dataset saved as: FABP4_06_bioactivity_data_3class_pIC50_rdkit_fp.csv
Dataset shape: (268, 2059)
Number of features: 2057
